# Домашнее задание 1

In [8]:
import os
import re
from pymystem3 import Mystem
import string
import nltk
import numpy as np
import pandas as pd
from datetime import datetime

## Коллекция

In [76]:
mystem = Mystem()

In [77]:
string.punctuation = string.punctuation.replace('-','')

In [85]:
def preprocessing(text):
    text = re.sub('(Автор|Регион):.*?\n','', text)
    text = re.sub('[0-9\n]','', text)
    text = re.sub('['+string.punctuation+']', '', text)
    text = ''.join(mystem.lemmatize(text.strip()))
    return text

In [86]:
data = pd.DataFrame(columns=['file', 'text'])

In [87]:
path = '.\\data\\'

In [88]:
start = datetime.now()
for root, dirs, files in os.walk (path):
    for fl in files:
        #print (fl)
        with open(path+fl, 'r', encoding = 'utf-8') as f:
            text = f.read()
        data = data.append({'file': fl, 'text': preprocessing(text)}, ignore_index=True)
print ('Total time: {}'.format(datetime.now()-start))

Total time: 0:31:43.948500


In [102]:
data.head()

,file,text
0,100_10.txt,известие московский выпуск трансляция суде...
1,100_2.txt,российский газета депутат продолжать...
2,100_9.txt,известие илья пономарев грозить арест имущ...
3,101_1.txt,известие москва дмитрий медведев менять си...
4,101_10.txt,коммерсант начинать первый уголовный пресле...


In [ ]:
import pickle as pkl

In [99]:
with open('data.pkl', 'wb') as f:
    pkl.dump(data, f)

In [101]:
with open('data.pkl', 'rb') as f:
    data = pkl.load(f)

In [96]:
def tabs(x):
    return re.sub('\t','', x)

In [97]:
data['text'] = data['text'].apply(tabs)

In [103]:
def load_frequency():
    freq_dict = {}
    with open ('.//freqrnc2011.csv','r', encoding='utf-8') as f:
        w = f.readlines()
    for i in w:
        s = i.strip().split('\t')
        if s[0] not in freq_dict:
            freq_dict[s[0].lower()] = float(s[1])
    return freq_dict
freq_dict = load_frequency()

In [104]:
print (freq_dict['я'])

12684.4


In [105]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [110]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                   stop_words=nltk.corpus.stopwords.words('russian'),
                                   analyzer ='word'
                                  )

In [111]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text'])

In [112]:
with open('tfidf_matrix.pkl', 'wb') as f:
    pkl.dump(tfidf_matrix, f)

In [113]:
tfidf_matrix.shape

(1228, 38314)

In [119]:
tfidf_vectorizer.idf_

array([6.72766175, 7.01534382, 7.42080893, ..., 4.85585957, 7.42080893,
       6.72766175])

In [120]:
tfidf_vectorizer.idf_.shape

(38314,)

## TF_IDF матрица для первых 5 документов

In [139]:
matrix_5 = tfidf_vectorizer.transform(data['text'][0:5], copy=True)

In [142]:
data_5 = pd.DataFrame(matrix_5.todense())

In [143]:
data_5.to_csv('1.csv', sep='\t')

In [144]:
data_5.columns

RangeIndex(start=0, stop=38314, step=1)

In [145]:
data_5.columns = tfidf_vectorizer.get_feature_names()

In [154]:
data_5.loc[:, (data_5 != 0).any(axis=0)]

,ddos,inbister,nirvana,spice,spravsibru,sro,ава,август,австралия,австрия,...,юридический,юрий,юрисдикция,юстиция,являться,якобы,яковлев,яна,январь,япония
0,0.059466,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019412,0.000000,0.000000,...,0.018047,0.000000,0.000000,0.000000,0.012010,0.000000,0.028119,0.02928,0.00000,0.000000
1,0.000000,0.052041,0.000000,0.000000,0.000000,0.052041,0.049198,0.026172,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.032383,0.000000,0.000000,0.00000,0.00000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.045028,0.000000,0.000000,0.023953,0.000000,0.000000,...,0.000000,0.065728,0.000000,0.000000,0.029638,0.022087,0.000000,0.00000,0.00000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.083856,0.037953,0.000000,0.000000,0.000000,0.00000,0.02453,0.000000
4,0.000000,0.000000,0.127669,0.127669,0.000000,0.000000,0.000000,0.000000,0.030631,0.033327,...,0.000000,0.000000,0.000000,0.000000,0.026481,0.000000,0.000000,0.00000,0.00000,0.026309


In [155]:
data_5.loc[:, (data_5 != 0).any(axis=0)].to_csv('1.csv', sep='\t')

## Ключевые слова

Пусть первый текст из коллекции

In [165]:
with open ('.\\data\\100_10.txt','r', encoding = 'utf-8') as f:
    text = f.read()
    print (re.sub('[\n]+', '\n', text))

 Известия 
 Автор: Герман Петелин 
   Генпрокуратура обжалует смягчение приговора педофилу Куваеву
"В понедельник Генпрокуратура внесет протест в Верховный суд на приговор педофилу Куваеву, который был смягчен", -заявил "Известиям" источник в надзорном ведомстве. После протеста суд может пересмотреть приговор скандально известному 40-летнему педофилу и "интернет-спамеру № 2 в мире" Леониду Куваеву. В марте этого года он получил 20 лет лишения свободы за изнасилование несовершеннолетних воспитанниц детдомов. После чего Верховный суд "снял" с него 20 эпизодов из 35 и снизил ему срок вдвое. Генпрокуратура сочла это решение слишком мягким. 
Судебный процесс над Леонидом Куваевым стал самым громким за последние годы. Его считали вторым по известности интернет-спамером в мире. За миллионером и организатором многих DDoS-атак безуспешно охотились американские спецслужбы, однако он попался в России на совращении и изнасиловании детей. 
Суд установил, что Куваев знакомился с детьми, представляяс

In [166]:
text = data['text'][0]

In [167]:
from collections import Counter

In [222]:
all_words = Counter(text.split())

In [197]:
matrix_1 = tfidf_vectorizer.transform([data['text'][0]], copy=True)
data_1 = pd.DataFrame(matrix_1.todense())
data_1.columns = tfidf_vectorizer.get_feature_names()

In [212]:
data_1.loc[:, (data_1 > 0.06).any(axis=0)]

,аудио,верховный,видеозапись,воспитанница,генпрокуратура,год,головко,девочка,депутат,закон,...,следователь,снисхождение,спамер,стенографирование,суд,судебный,судья,трансляция,требовать,эпизод
0,0.09233,0.066208,0.07303,0.06999,0.07303,0.092568,0.06999,0.156834,0.061243,0.086213,...,0.064661,0.06999,0.090686,0.06999,0.236621,0.108586,0.062468,0.093692,0.061125,0.099491


In [269]:
data_1['закон'][0]

0.08621309501763894

In [198]:
words = data_1.loc[:, (data_1 > 0.06).any(axis=0)].columns
words

Index(['аудио', 'верховный', 'видеозапись', 'воспитанница', 'генпрокуратура',
       'год', 'головко', 'девочка', 'депутат', 'закон', 'законопроект',
       'заседание', 'игошин', 'известие', 'изнасилование', 'куваев', 'куваева',
       'леонид', 'малолетний', 'мвд', 'несовершеннолетний', 'оргия', 'педофил',
       'половой', 'приводить', 'приговор', 'проституция', 'ребенок', 'россия',
       'следователь', 'снисхождение', 'спамер', 'стенографирование', 'суд',
       'судебный', 'судья', 'трансляция', 'требовать', 'эпизод'],
      dtype='object')

In [199]:
def freq(word):
    if word in freq_dict:
        return freq_dict[word]
    else:
        return 0

freqs = [freq(word) for word in words]

In [210]:
#tf_idf = 
tf_idfs = [data_1.iloc(0)[0][data_1.iloc(0)[0]>0][word] for word in words]

In [232]:
total_text = len(text.split())
total_text

1240

In [243]:
tf = [all_words[word]/total_text*1000000 for word in words]

In [234]:
total = len(" ".join(data["text"]).split())
total

764915

In [235]:
df_all = Counter(" ".join(data["text"]).split())

In [236]:
len(df_all)

45613

In [244]:
collection = [df_all[word]/total*1000000 for word in words]

In [250]:
key_words = pd.DataFrame({'word':words, 
                          'ipm_freq':freqs, 
                          'tfidf':tf_idfs, 
                          'ipm_tf':tf, 
                          'ipm_collection':collection, 
                          'abs_tf':np.array(tf)/1000000})
key_words = key_words.sort_values(by=['tfidf'], ascending=False)
key_words.to_csv('HW1_Glazunov_KeyWords.txt', index=False, sep='\t')

In [251]:
key_words

,abs_tf,ipm_collection,ipm_freq,ipm_tf,tfidf,word
15,0.012903,70.596079,0.0,12903.225806,0.513342,куваев
22,0.007258,70.596079,1.0,7258.064516,0.247216,педофил
33,0.013710,943.895727,371.1,13709.677419,0.236621,суд
16,0.004839,19.610022,0.0,4838.709677,0.209969,куваева
7,0.005645,141.192159,185.1,5645.161290,0.156834,девочка
34,0.004839,271.925639,126.1,4838.709677,0.108586,судебный
11,0.004839,270.618304,96.7,4838.709677,0.104079,заседание
38,0.003226,48.371388,43.5,3225.806452,0.099491,эпизод
27,0.005645,913.827027,658.3,5645.161290,0.094396,ребенок
36,0.003226,70.596079,7.5,3225.806452,0.093692,трансляция


## Редкие слова в коллекции

In [ ]:
df_all.items()

In [287]:
rare_words = [key for key, value in sorted(df_all.items(), key = lambda x: x[1]) if value<4 and key in all_words]

In [288]:
rare_words = [word for word in rare_words if all_words[word] < 2]
rare_words

['оппозицииизвестие',
 'куваевув',
 'развратник',
 'сводница',
 'худенький',
 'жуков',
 'невменяемость',
 'педофильский',
 'сношение',
 'гособвинение',
 'общедоступнымипроект',
 'государственный-правовой',
 'поддавливать',
 'неразборчиво',
 'натравлять',
 '-заявлять',
 'втираться',
 'водить',
 'утеха',
 'пособничество',
 'сауна',
 'джакузи',
 'барный',
 'приглушенный',
 'прения',
 'массачусетс']

## Разделение коллекции

In [295]:
new_texts = [" ".join(data["text"][i*122 : i*122 + 122]) for i in range(0,10)]

In [ ]:
new_texts